In [3]:
using Korg, ProgressMeter, HDF5, PyPlot

In [2]:
atmosphere_archive = "SDSS_MARCS_atmospheres.h5"

"SDSS_MARCS_atmospheres.h5"

In [4]:
atm_dir = "/uufs/chpc.utah.edu/common/home/sdss50/dr17/apogee/spectro/speclib/atmos/marcs/MARCS_v3_2016"
subdirs = ["mod_z+0.00" ,"mod_z+0.25" ,"mod_z-0.25" ,"mod_z+0.50" ,"mod_z-0.50" ,"mod_z+0.75" ,"mod_z-0.75" ,"mod_z+1.00" ,"mod_z-1.00" ,"mod_z-1.25" ,"mod_z-1.50" ,"mod_z-1.75" ,"mod_z-2.00" ,"mod_z-2.25" ,"mod_z-2.50"]
#subdirs = ["mod_z+0.00"]
;

In [6]:
h5open("SDSS_MARCS_atmospheres.h5", "r+") do h
    println(read_attribute(h["version"]))
end

LoadError: KeyError: key "version" not found

# old: pack with separation between spherical and planar 

In [15]:
#this order is different and important
interpolation_params = ["Teff", "logg", "metallicity", "alpha", "carbon"]
#filename_regex = r"^([ps])(.*)_g(.*)_m(.*)_t(.*)_..?_z(.*)_a(.*)_c(.*)_n(.*)_o(.*)_r(.*)_s(.*).mod(.filled)?$"
filename_regex = r"^([ps])(.*)_g(.*)_m(.*)_t(.*)_..?_z(.*)_a(.*)_c(.*)_n(.*)_o(.*)_r(.*)_s(.*).mod$"
capture_inds = [2, 3, 6, 7, 8]

h5open(atmosphere_archive, "w") do h
    h["grid_parameter_names"] = interpolation_params
    
    #first get the set of all possible values
    all_vals_spherical = [[] for p in interpolation_params]
    all_vals_planar = [[] for p in interpolation_params]
    println("determining node values...")
    @showprogress for subdir in subdirs, filename in readdir(joinpath(atm_dir, subdir))
        m = match(filename_regex, filename)
        if !isnothing(m)
            vals = m.captures[capture_inds]
            if m.captures[1] == "s"
                push!.(all_vals_spherical, vals)
            else
                push!.(all_vals_planar, vals)
            end
        end
    end
    grid_vals_spherical = [sort(parse.(Float32, (collect(Set(vals))))) for vals in all_vals_spherical]
    grid_vals_planar = [sort(parse.(Float32, (collect(Set(vals))))) for vals in all_vals_planar]
    for i in eachindex(interpolation_params)
        h["spherical/grid_values/$i"] = grid_vals_spherical[i]
        h["planar/grid_values/$i"] =  grid_vals_planar[i]
    end
    
    spherical_exists = zeros(Bool, Tuple(length.(grid_vals_spherical)))
    spherical_grid = zeros(Float32, (56, 5, length.(grid_vals_spherical)...))
    R_grid = zeros(Float32, Tuple(length.(grid_vals_spherical)))
    planar_exists = zeros(Bool, Tuple(length.(grid_vals_planar)))
    planar_grid = zeros(Float32, (56, 5, length.(grid_vals_planar)...))
    
    println("loading and packing atmospheres...")
    @showprogress for subdir in subdirs, filename in readdir(joinpath(atm_dir, subdir))
        m = match(filename_regex, filename)
        isnothing(m) && continue #skip if it's not a standard composition .mod file
        if !(m.captures[4] in ["1.0", "0.0"])
            continue #skip if mass != 1
        end
        
        atm = read_model_atmosphere(joinpath(atm_dir, subdir, filename))
       
        vals = parse.(Float32, m.captures[capture_inds])
        grid, exists, grid_vals = if m.captures[1] == "s" 
            spherical_grid, spherical_exists, grid_vals_spherical
        else
            planar_grid, planar_exists, grid_vals_planar
        end
        inds = [findfirst(v .== gv) for (v, gv) in zip(vals, grid_vals)]
        m.captures[1] == "s" && (R_grid[inds...] = atm.R)
        exists[inds...] = true
        grid[:, 1, inds...] = [l.temp for l in atm.layers]
        grid[:, 2, inds...] = [log(l.electron_number_density) for l in atm.layers]
        grid[:, 3, inds...] = [log(l.number_density) for l in atm.layers]
        grid[:, 4, inds...] = [l.tau_5000 for l in atm.layers]
        grid[:, 5, inds...] = [asinh(l.z) for l in atm.layers]
    end
    
    h["spherical/exists"] = spherical_exists
    h["spherical/grid", shuffle=(), deflate=9] = spherical_grid
    h["spherical/R_grid", shuffle=(), deflate=9] = R_grid
    h["planar/exists"] = planar_exists
    h["planar/grid", shuffle=(), deflate=9] = planar_grid
    
    ;
end
;

determining node values...


Progress: 100%|█████████████████████████████████████████| Time: 0:00:13


loading and packing atmospheres...


Progress: 100%|█████████████████████████████████████████| Time: 0:07:01


In [29]:
spherical_exists, spherical_nodes, planar_exists, planar_nodes = h5open(atmosphere_archive) do h
    read(h["spherical/exists"]), read(h["spherical/grid_values"]), read(h["planar/exists"]), read(h["planar/grid_values"])
end
spherical_nodes = [spherical_nodes[string(i)] for i in 1:length(spherical_nodes)]
planar_nodes = [planar_nodes[string(i)] for i in 1:length(planar_nodes)]
;

# new: pack planar and spherical together

In [ ]:
#this order is different and important
interpolation_params = ["Teff", "logg", "metallicity", "alpha", "carbon"]
filename_regex = r"^([ps])(.*)_g(.*)_m(.*)_t(.*)_..?_z(.*)_a(.*)_c(.*)_n(.*)_o(.*)_r(.*)_s(.*).mod(.filled)?$"
#filename_regex = r"^([ps])(.*)_g(.*)_m(.*)_t(.*)_..?_z(.*)_a(.*)_c(.*)_n(.*)_o(.*)_r(.*)_s(.*).mod$"
capture_inds = [2, 3, 6, 7, 8]

h5open(atmosphere_archive, "w") do h
    h["grid_parameter_names"] = interpolation_params
    
    #first get the set of all possible values
    all_vals = [[] for p in interpolation_params]
    println("determining node values...")
    @showprogress for subdir in subdirs, filename in readdir(joinpath(atm_dir, subdir))
        m = match(filename_regex, filename)
        if !isnothing(m)
            vals = m.captures[capture_inds]
            push!.(all_vals, vals)
        end
    end
    grid_vals = [sort(parse.(Float32, (collect(Set(vals))))) for vals in all_vals]
    for i in eachindex(interpolation_params)
        h["grid_values/$i"] = grid_vals[i]
    end
    
    exists = zeros(Bool, Tuple(length.(grid_vals)))
    grid = zeros(Float32, (56, 5, length.(grid_vals)...))
    
    println("loading and packing atmospheres...")
    @showprogress for subdir in subdirs, filename in readdir(joinpath(atm_dir, subdir))
        m = match(filename_regex, filename)
        isnothing(m) && continue #skip if it's not a standard composition .mod file
        if !(m.captures[4] in ["1.0", "0.0"])
            continue #skip if mass != 1
        end
        
        atm = read_model_atmosphere(joinpath(atm_dir, subdir, filename))
       
        vals = parse.(Float32, m.captures[capture_inds])
        inds = [findfirst(v .== gv) for (v, gv) in zip(vals, grid_vals)]
        exists[inds...] = true
        grid[:, 1, inds...] = [l.temp for l in atm.layers]
        grid[:, 2, inds...] = [log(l.electron_number_density) for l in atm.layers]
        grid[:, 3, inds...] = [log(l.number_density) for l in atm.layers]
        grid[:, 4, inds...] = [l.tau_5000 for l in atm.layers]
        grid[:, 5, inds...] = [asinh(l.z) for l in atm.layers]
    end
    
    h["exists"] = exists
    h["grid", shuffle=(), deflate=9] = grid
end
;

determining node values...


Progress: 100%|█████████████████████████████████████████| Time: 0:00:10


loading and packing atmospheres...


Progress:  60%|████████████████████████▋                |  ETA: 0:02:46

In [8]:
h5open(atmosphere_archive) do h
    display(sum(read(h["exists"])))
end

579150

In [16]:
h5open(atmosphere_archive) do h
    display(sum(read(h["exists"])))
end

470539

In [29]:
exists, nodes = h5open(atmosphere_archive) do h
    read(h["spherical/exists"]), read(h["spherical/grid_values"]), read(h["planar/exists"]), read(h["planar/grid_values"])
end
spherical_nodes = [spherical_nodes[string(i)] for i in 1:length(spherical_nodes)]
planar_nodes = [planar_nodes[string(i)] for i in 1:length(planar_nodes)]
;

In [7]:
axs = subplots(nrows=5, ncols=5, figsize=(15,15))[2]
[ax.set_visible(false) for ax in axs]

names = ["teff", "logg", "[Fe/H]", "[α/H]", "[C/H]"]

#for (exists, nodes) in [(spherical_exists, spherical_nodes), (planar_exists, planar_nodes)]
#let exists=spherical_exists, nodes=spherical_nodes
#let exists=planar_exists, nodes=planar_nodes
rows = []
for I in CartesianIndices(exists)
    if exists[I]
        push!(rows, getindex.(nodes, collect(Tuple(I))))
    end
end

M = vcat(transpose.(rows)...)

for i in 1:length(names), j in 1:i-1
    sca(axs[i,j])
    gca().set_visible(true)
    scatter(M[:, j], M[:, i], color="k", s=1)
    ylabel(names[i])
    xlabel(names[j])
end

tight_layout()

LoadError: UndefVarError: exists not defined